In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from tqdm import tqdm

In [ ]:
# Load the full data
data = pd.read_parquet("/content/drive/MyDrive/LSE Capstone G Drive/Handling Missing Data/5km resolution data/3.5 years data(2022-2025)/5km_resolution_with_missing_data_handled_2022_to_2025.parquet", engine='pyarrow')

# Show shape and columns
print("Shape:", data.shape)
print("Columns:", data.columns.tolist())
print(data.head())

Shape: (14146902, 13)
Columns: ['lat_5km', 'lon_5km', 'KD490', 'ZSD', 'RRS490', 'RRS443', 'CHL', 'MICRO', 'BBP', 'CDM', 'time', 'flags', 'date']
   lat_5km  lon_5km     KD490        ZSD  RRS490  RRS443       CHL  MICRO  \
0    49.15   -10.75  0.035727  26.936846    -2.0    -2.0  0.124980   -2.0   
1    49.15   -10.70  0.035727  26.936848    -2.0    -2.0  0.124980   -2.0   
2    49.15   -10.65  0.035787  26.878199    -2.0    -2.0  0.125559   -2.0   
3    49.15   -10.60  0.035876  26.790228    -2.0    -2.0  0.126427   -2.0   
4    49.15   -10.55  0.035876  26.790228    -2.0    -2.0  0.126427   -2.0   

   BBP  CDM       time  flags        date  
0 -2.0 -2.0 2025-01-01    0.0  2025-01-01  
1 -2.0 -2.0 2025-01-01    0.0  2025-01-01  
2 -2.0 -2.0 2025-01-01    0.0  2025-01-01  
3 -2.0 -2.0 2025-01-01    0.0  2025-01-01  
4 -2.0 -2.0 2025-01-01    0.0  2025-01-01  


# Data Preprocessing

## Remove Land Data (-1) and Missing Data during winter months (-2)

In [ ]:
# Define parameter columns that might contain -1.0 for land data
parameter_columns = ["KD490", "ZSD", "RRS490", "RRS443", "CHL", "MICRO", "BBP", "CDM"]

# Filter out rows where "flags" column is 1.0 (land data)
data = data[data["flags"] != 1.0].copy()

# Filter out rows where any parameter column contains -1.0 (land data)
land_data_in_params = (data[parameter_columns] == -1.0).any(axis=1)
data = data[~land_data_in_params].copy()

data = data.drop(['time', 'flags'], axis=1)
print(f"Shape after removing land data: {data.shape}")

Shape after removing land data: (10987584, 11)


In [ ]:
# Replace -2.0 values in parameter columns with NaN
for col in parameter_columns:
    data[col] = data[col].replace(-2.0, np.nan)

print("Missing data (-2.0) replaced with NaN.")
print("NaN counts per parameter column after replacement:")
print(data[parameter_columns].isnull().sum())

# Note: Interpolation of NaNs will be handled after daily aggregation
# to ensure continuity in the time series for modeling.

Missing data (-2.0) replaced with NaN.
NaN counts per parameter column after replacement:
KD490           0
ZSD             0
RRS490    1443936
RRS443    1443936
CHL             0
MICRO     1871136
BBP       1666080
CDM       1666080
dtype: int64


In [ ]:
initial_rows = data.shape[0]
print(f"Initial Data Shape: {data.shape}")

print("NaN counts per parameter column before dropping NaNs:")
print(data[parameter_columns].isnull().sum())

# Remove NaN values
data.dropna(inplace=True)
final_rows = data.shape[0]
print(f"\nRemoved {initial_rows - final_rows} rows containing NaN values.")
print(f"New Data Shape after dropping NaNs: {data.shape}")

Initial Data Shape: (10987584, 11)
NaN counts per parameter column before dropping NaNs:
KD490           0
ZSD             0
RRS490    1443936
RRS443    1443936
CHL             0
MICRO     1871136
BBP       1666080
CDM       1666080
dtype: int64

Removed 1871136 rows containing NaN values.
New Data Shape after dropping NaNs: (9116448, 11)


In [ ]:
data.to_parquet("/content/drive/MyDrive/LSE Capstone G Drive/Train Val Test Sets/2D Dataframes/2d_data_cleaned.parquet")
data

,lat_5km,lon_5km,KD490,ZSD,RRS490,RRS443,CHL,MICRO,BBP,CDM,date
272475,49.15,-10.75,0.042622,21.520136,0.006361,0.006838,0.197671,0.285306,0.003873,0.016502,2025-01-26
272476,49.15,-10.70,0.043422,21.033817,0.007095,0.007484,0.206785,0.285306,0.003873,0.016502,2025-01-26
272477,49.15,-10.65,0.043873,20.774261,0.005769,0.006020,0.212015,0.285306,0.003873,0.016502,2025-01-26
272478,49.15,-10.60,0.043365,21.072351,0.003745,0.004274,0.206158,0.285306,0.003873,0.016502,2025-01-26
272479,49.15,-10.55,0.044909,20.184818,0.003184,0.004002,0.224094,0.285306,0.003873,0.016502,2025-01-26
...,...,...,...,...,...,...,...,...,...,...,...
13667298,52.25,-4.50,0.135265,5.545503,0.005816,0.006983,1.893622,0.266766,0.003325,0.015229,2024-11-17
13667299,52.25,-4.45,0.135620,5.536804,0.005816,0.006983,1.903060,0.266766,0.003325,0.015229,2024-11-17
13667300,52.25,-4.40,0.130535,5.786188,0.005816,0.006983,1.782744,0.266766,0.003325,0.015229,2024-11-17
13667301,52.25,-4.35,0.141437,5.263031,0.005816,0.006983,2.042302,0.266766,0.003325,0.015229,2024-11-17


## Creation of Train / Validation / Test sets

### Identify date ranges of the data & check for missing dates

In [ ]:
# Convert to datetime if needed
data['date'] = pd.to_datetime(data['date'])

# Unique sorted dates
unique_dates = pd.to_datetime(sorted(data['date'].unique()))
print(f"Number of unique dates: {len(unique_dates)}")
print(f"Date range: {unique_dates.min().date()} to {unique_dates.max().date()}")

# Full expected date range (daily)
expected_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='D')

# Find missing dates
missing_dates = set(expected_dates) - set(unique_dates)
if missing_dates:
    print(f"\nMissing dates: {len(missing_dates)}")
    print("Example missing dates:", sorted(missing_dates)[:10])
else:
    print("\n All dates are continuous.")

print(sorted(missing_dates))

Number of unique dates: 1067
Date range: 2022-01-27 to 2025-07-21

Missing dates: 205
Example missing dates: [Timestamp('2022-11-21 00:00:00'), Timestamp('2022-11-22 00:00:00'), Timestamp('2022-11-23 00:00:00'), Timestamp('2022-11-24 00:00:00'), Timestamp('2022-11-25 00:00:00'), Timestamp('2022-11-26 00:00:00'), Timestamp('2022-11-27 00:00:00'), Timestamp('2022-11-28 00:00:00'), Timestamp('2022-11-29 00:00:00'), Timestamp('2022-11-30 00:00:00')]
[Timestamp('2022-11-21 00:00:00'), Timestamp('2022-11-22 00:00:00'), Timestamp('2022-11-23 00:00:00'), Timestamp('2022-11-24 00:00:00'), Timestamp('2022-11-25 00:00:00'), Timestamp('2022-11-26 00:00:00'), Timestamp('2022-11-27 00:00:00'), Timestamp('2022-11-28 00:00:00'), Timestamp('2022-11-29 00:00:00'), Timestamp('2022-11-30 00:00:00'), Timestamp('2022-12-01 00:00:00'), Timestamp('2022-12-02 00:00:00'), Timestamp('2022-12-03 00:00:00'), Timestamp('2022-12-04 00:00:00'), Timestamp('2022-12-05 00:00:00'), Timestamp('2022-12-06 00:00:00'), Times

### Create and save the sets in tabular 2D arrays

In [ ]:
# Calculate sizes for data split
train_start = pd.to_datetime("2022-01-27")
train_end = pd.to_datetime("2023-11-17")
val_start = pd.to_datetime("2024-01-26")
val_end = pd.to_datetime("2024-07-31")
test_start = pd.to_datetime("2024-08-01")
test_end = pd.to_datetime("2025-07-21")

# Filter data for each split
train_set = data[(data["date"]>= train_start) & (data["date"]<= train_end)]
val_set = data[(data["date"]>= val_start) & (data["date"]<= val_end)]
test_set = data[(data["date"]>= test_start) & (data["date"]<= test_end)]

print(f"Train set dates: {train_set['date'].min().date()} to {train_set['date'].max().date()}")
print(f"Validation set dates: {val_set['date'].min().date()} to {val_set['date'].max().date()}")
print(f"Test set dates: {test_set['date'].min().date()} to {test_set['date'].max().date()}")

# Ensure data is sorted, Find positional indices of the split boundaries
unique_train_days = pd.to_datetime(sorted(train_set["date"].unique()))
unique_val_days = pd.to_datetime(sorted(val_set["date"].unique()))
unique_test_days = pd.to_datetime(sorted(test_set["date"].unique()))

print(f"\nTotal number of unique days: {len(unique_dates)}")
print(f"\nTrain days: {len(unique_train_days)}")
print(f"Val days: {len(unique_val_days)}")
print(f"Test days: {len(unique_test_days)}")
print(f"Sum of days: {len(unique_train_days) + len(unique_val_days) + len(unique_test_days)}")

train_set.to_parquet("train_set_2D.parquet")
val_set.to_parquet("val_set_2D.parquet")
test_set.to_parquet("test_set_2D.parquet")

Train set dates: 2022-01-27 to 2023-11-17
Validation set dates: 2024-01-26 to 2024-07-31
Test set dates: 2024-08-01 to 2025-07-21

Total number of unique days: 1067

Train days: 593
Val days: 188
Test days: 286
Sum of days: 1067


### Find the continuous blocks of data (winter gaps in between) in train-val-test sets

In [ ]:
def find_blocks(df):
    dates = pd.to_datetime(sorted(df['date'].unique()))
    blocks = []

    start_date = dates[0]
    prev_date = dates[0]

    for current_date in dates[1:]:
        # If there’s a gap >1 day, close the current block
        if (current_date - prev_date).days > 1:
            blocks.append((start_date.date(), prev_date.date()))
            start_date = current_date
        prev_date = current_date

    # Append the last block
    blocks.append((start_date.date(), prev_date.date()))
    return blocks

train_blocks = find_blocks(train_set)
val_blocks = find_blocks(val_set)
test_blocks = find_blocks(test_set)

print("Train Set Blocks:")
for b in train_blocks:
    print(f"{b[0]}  →  {b[1]}")

print("\nValidation Set Blocks:")
for b in val_blocks:
    print(f"{b[0]}  →  {b[1]}")

print("\nTest Set Blocks:")
for b in test_blocks:
    print(f"{b[0]}  →  {b[1]}")

Train Set Blocks:
2022-01-27  →  2022-11-20
2023-01-27  →  2023-11-17

Validation Set Blocks:
2024-01-26  →  2024-07-31

Test Set Blocks:
2024-08-01  →  2024-11-17
2025-01-26  →  2025-07-21


### Sanity check for missing periods of data overall (not used in modelling)
This is to make sure that the only missing data indeed falls within the winter months for the train set, and the validation set should have no missing data at all.

In [ ]:
# --- Check for missing dates in train set ---
expected_train_dates = pd.date_range(start=train_set['date'].min(), end=train_set['date'].max(), freq='D')
actual_train_dates = pd.to_datetime(train_set['date'].unique())
missing_train_dates = expected_train_dates.difference(actual_train_dates)

print(f"\nTrain set expected {len(expected_train_dates)} days, actual {len(actual_train_dates)} days.")
print(f"Missing train dates: {missing_train_dates.tolist()}" if len(missing_train_dates) > 0 else "No missing dates in train set.")

# --- Check for missing dates in validation set ---
expected_val_dates = pd.date_range(start=val_set['date'].min(), end=val_set['date'].max(), freq='D')
actual_val_dates = pd.to_datetime(val_set['date'].unique())
missing_val_dates = expected_val_dates.difference(actual_val_dates)

print(f"\nValidation set expected {len(expected_val_dates)} days, actual {len(actual_val_dates)} days.")
print(f"Missing validation dates: {missing_val_dates.tolist()}" if len(missing_val_dates) > 0 else "No missing dates in validation set.")


Train set expected 660 days, actual 593 days.
Missing train dates: [Timestamp('2022-11-21 00:00:00'), Timestamp('2022-11-22 00:00:00'), Timestamp('2022-11-23 00:00:00'), Timestamp('2022-11-24 00:00:00'), Timestamp('2022-11-25 00:00:00'), Timestamp('2022-11-26 00:00:00'), Timestamp('2022-11-27 00:00:00'), Timestamp('2022-11-28 00:00:00'), Timestamp('2022-11-29 00:00:00'), Timestamp('2022-11-30 00:00:00'), Timestamp('2022-12-01 00:00:00'), Timestamp('2022-12-02 00:00:00'), Timestamp('2022-12-03 00:00:00'), Timestamp('2022-12-04 00:00:00'), Timestamp('2022-12-05 00:00:00'), Timestamp('2022-12-06 00:00:00'), Timestamp('2022-12-07 00:00:00'), Timestamp('2022-12-08 00:00:00'), Timestamp('2022-12-09 00:00:00'), Timestamp('2022-12-10 00:00:00'), Timestamp('2022-12-11 00:00:00'), Timestamp('2022-12-12 00:00:00'), Timestamp('2022-12-13 00:00:00'), Timestamp('2022-12-14 00:00:00'), Timestamp('2022-12-15 00:00:00'), Timestamp('2022-12-16 00:00:00'), Timestamp('2022-12-17 00:00:00'), Timestamp('20